In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, TimeDistributed, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# -------------------------------
# Step 1: Load & Preprocess Dataset
# -------------------------------

DATASET_PATH = "/kaggle/input/sign-language-recognition"  # Your Kaggle dataset path
IMG_SIZE = 64  # Resize images to 64x64
SEQUENCE_LENGTH = 10  # Number of frames per sequence
NUM_CLASSES = 10  # Adjust based on the number of sign classes

def load_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while len(frames) < SEQUENCE_LENGTH:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))  # Resize to 64x64
        frames.append(frame)
    
    cap.release()
    
    # If video has fewer frames, pad with last frame
    while len(frames) < SEQUENCE_LENGTH:
        frames.append(frames[-1])
    
    return np.array(frames)

# Load dataset
X, y = [], []
labels = os.listdir(DATASET_PATH)  # Each folder is a label (A, B, C, etc.)

label_dict = {label: i for i, label in enumerate(labels)}  # Encode labels

for label in labels:
    label_path = os.path.join(DATASET_PATH, label)
    for video in os.listdir(label_path):
        video_path = os.path.join(label_path, video)
        frames = load_video_frames(video_path)
        X.append(frames)
        y.append(label_dict[label])

# Convert to NumPy arrays
X = np.array(X).reshape(-1, SEQUENCE_LENGTH, IMG_SIZE, IMG_SIZE, 1)  # Reshape for CNN input
y = to_categorical(y, num_classes=NUM_CLASSES)  # One-hot encode labels

# Split dataset into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -------------------------------
# Step 2: Build CNN-LSTM Model
# -------------------------------

model = Sequential([
    TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'), input_shape=(SEQUENCE_LENGTH, IMG_SIZE, IMG_SIZE, 1)),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Flatten()),
    LSTM(128, return_sequences=False),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

# -------------------------------
# Step 3: Train the Model
# -------------------------------

EPOCHS = 20
BATCH_SIZE = 16

history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test))

# -------------------------------
# Step 4: Evaluate the Model
# -------------------------------

loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc * 100:.2f}%")

# -------------------------------
# Step 5: Save the Model
# -------------------------------

model.save("/kaggle/working/sign_language_cnn_lstm.h5")
print("Model saved successfully!")


NotADirectoryError: [Errno 20] Not a directory: '/kaggle/input/sign-language-recognition/__results__.html'

In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, TimeDistributed, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# -------------------------------
# Step 1: Load & Preprocess Dataset
# -------------------------------

DATASET_PATH = "/kaggle/input/sign-language-recognition"  # Your Kaggle dataset path
IMG_SIZE = 64  # Resize images to 64x64
SEQUENCE_LENGTH = 10  # Number of frames per sequence

# Get only label directories, ignore files
labels = [d for d in os.listdir(DATASET_PATH) if os.path.isdir(os.path.join(DATASET_PATH, d))]
NUM_CLASSES = len(labels)  # Dynamically set the number of classes

label_dict = {label: i for i, label in enumerate(labels)}  # Encode labels

def load_video_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while len(frames) < SEQUENCE_LENGTH:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))  # Resize to 64x64
        frames.append(frame)
    
    cap.release()
    
    # If video has fewer frames, pad with last frame
    while len(frames) < SEQUENCE_LENGTH:
        frames.append(frames[-1])
    
    return np.array(frames)

# Load dataset
X, y = [], []

for label in labels:
    label_path = os.path.join(DATASET_PATH, label)
    if not os.path.isdir(label_path):  # Ensure it's a directory
        continue
    
    for video in os.listdir(label_path):
        video_path = os.path.join(label_path, video)
        if not video_path.endswith(('.mp4', '.avi', '.mov')):  # Only process video files
            continue
        
        frames = load_video_frames(video_path)
        X.append(frames)
        y.append(label_dict[label])

# Convert to NumPy arrays
X = np.array(X).reshape(-1, SEQUENCE_LENGTH, IMG_SIZE, IMG_SIZE, 1)  # Reshape for CNN input
y = to_categorical(y, num_classes=NUM_CLASSES)  # One-hot encode labels

# Split dataset into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -------------------------------
# Step 2: Build CNN-LSTM Model
# -------------------------------

model = Sequential([
    TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'), input_shape=(SEQUENCE_LENGTH, IMG_SIZE, IMG_SIZE, 1)),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Flatten()),
    LSTM(128, return_sequences=False),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

# -------------------------------
# Step 3: Train the Model
# -------------------------------

EPOCHS = 20
BATCH_SIZE = 16

history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test))

# -------------------------------
# Step 4: Evaluate the Model
# -------------------------------

loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc * 100:.2f}%")

# -------------------------------
# Step 5: Save the Model
# -------------------------------

model.save("/kaggle/working/sign_language_cnn_lstm.h5")
print("Model saved successfully!")


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, TimeDistributed, Flatten, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.utils import to_categorical

# تعريف نموذج CNN-LSTM
def create_cnn_lstm_model(input_shape, num_classes):
    model = Sequential()

    # طبقات CNN
    model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=input_shape))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
    model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
    model.add(TimeDistributed(Flatten()))

    # طبقات LSTM
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(64))
    model.add(Dropout(0.5))

    # طبقات Fully Connected
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    # تجميع النموذج
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# تحضير البيانات (هذا مثال، يجب استبداله ببياناتك الخاصة)
def load_example_data():
    # مثال: إنشاء بيانات وهمية
    num_samples = 100  # عدد العينات
    num_frames = 30    # عدد الإطارات في كل فيديو
    height, width, channels = 64, 64, 3  # أبعاد الإطارات
    num_classes = 10   # عدد الفئات (العلامات)

    X_train = np.random.rand(num_samples, num_frames, height, width, channels)  # بيانات تدريب
    y_train = np.random.randint(0, num_classes, num_samples)  # تسميات تدريب

    X_test = np.random.rand(num_samples // 2, num_frames, height, width, channels)  # بيانات اختبار
    y_test = np.random.randint(0, num_classes, num_samples // 2)  # تسميات اختبار

    # تحويل التسميات إلى one-hot encoding
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

    return X_train, X_test, y_train, y_test

# تحميل البيانات
X_train, X_test, y_train, y_test = load_example_data()

# تعريف شكل المدخلات وعدد الفئات
input_shape = (None, 64, 64, 3)  # (عدد الإطارات, ارتفاع, عرض, قنوات)
num_classes = 10

# إنشاء النموذج
model = create_cnn_lstm_model(input_shape, num_classes)

# تدريب النموذج
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# تقييم النموذج
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# حفظ النموذج
model.save('sign_language_cnn_lstm.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 98s 21s/step - accuracy: 0.0891 - loss: 2.4063 - val_accuracy: 0.1000 - val_loss: 2.3248
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 60s 14s/step - accuracy: 0.1307 - loss: 2.3275 - val_accuracy: 0.0400 - val_loss: 2.3226
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 61s 14s/step - accuracy: 0.1027 - loss: 2.3678 - val_accuracy: 0.0400 - val_loss: 2.3243
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 82s 14s/step - accuracy: 0.1089 - loss: 2.3903 - val_accuracy: 0.0400 - val_loss: 2.3177
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 81s 14s/step - accuracy: 0.1508 - loss: 2.2664 - val_accuracy: 0.0400 - val_loss: 2.3098
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 62s 14s/step - accuracy: 0.0983 - loss: 2.3159 - val_accuracy: 0.0200 - val_loss: 2.3067
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 61s 14s/step - accuracy: 0.1598 - loss: 2.2842 - val_accuracy: 0.1000 - val_loss: 2.3068
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 60s 14s/step - accuracy: 0.0930 - loss: 2.3849 - val_accuracy: 0.1000 - val_loss: 2.3073
